1. 라이브러리 및 모듈

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from tqdm import tqdm

2. Tic Tac Toe 환경 정의

In [2]:
class Environment():
   
    def __init__(self):
    # 보드는 0으로 초기화된 9개의 배열로 준비함
    # 게임종료 : done = True
#         self.board_a = np.zeros(9)
        self.board_a = np.zeros(16)
        self.done = False
        self.reward = 0
        self.winner = 0
        self.print = False
   

    def move(self, p1, p2, player):
    # 각 플레이어가 선택한 행동을 표시 하고 게임 상태(진행 또는 종료)를 판단
    # p1 = 1, p2 = -1로 정의
    # 각 플레이어는 행동을 선택하는 select_action 메서드를 가짐
        if player == 1:
            pos = p1.select_action(env, player)
        else:
            pos = p2.select_action(env, player)
            
       
        # 보드에 플레이어의 선택을 표시
        self.board_a[pos] = player
        if self.print:
            print(player)
            self.print_board()
        # 게임이 종료상태인지 아닌지를 판단
        self.end_check(player)
       
        return  self.reward, self.done
 
    # 현재 보드 상태에서 가능한 행동(둘 수 있는 장소)을 탐색하고 리스트로 반환
    def get_action(self):
        observation = []
        
        for i in range(16):
            if self.board_a[i] == 0:
                observation.append(i)
        return observation
   
    # 게임이 종료(승패 또는 비김)됐는지 판단
    def end_check(self,player):
        # 0   1   2   3
        # 4   5   6   7
        # 8   9  10  11
        #12  13  14  15
        
        # 4 x 4
        end_condition = [  
            # 가로
            (0,1,2,3), (4,5,6,7), (8,9,10,11), (12,13,14,15),
            # 세로
            (0,4,8,12), (1,5,9,13), (2,6,10,14), (3,7,11,15),
            # 대각선
            (0,5,10,15), (3,6,9,12)
        ]
        for line in end_condition:
             if self.board_a[line[0]] == self.board_a[line[1]] \
                and self.board_a[line[1]] == self.board_a[line[2]] \
                and self.board_a[line[2]] == self.board_a[line[3]] \
                and self.board_a[line[0]] != 0:  
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = player
                return
        # 비긴 상태는 더는 보드에 빈 공간이 없을때
        observation = self.get_action()
        if (len(observation)) == 0:
            self.done = True
            self.reward = 0            
        return
       
    # 현재 보드의 상태를 표시 p1 = O, p2 = X    
    def print_board(self): #4x4로 수정 완료
        print("+----+----+----+----+")  
        for i in range(4): # 줄 수 
            for j in range(4):  # 칸 수 
                if self.board_a[4*i+j] == 1:  #index 계산식도 4*i+j로 바꿈
                    print("|  O",end=" ")
                elif self.board_a[4*i+j] == -1:
                    print("|  X",end=" ")
                else:
                    print("|   ",end=" ")
            print("|")
            print("+----+----+----+----+")  # 출력 테두리 4칸용으로 바꿈 


3. Human player

In [3]:
class Human_player():
    
    def __init__(self):
        self.name = "Human player"
        
    def select_action(self, env, player):
        while True:
            # 가능한 행동을 조사한 후 표시
            available_action = env.get_action()
            print("possible actions = {}".format(available_action))

            # 4x4 상태 번호 표시로 수정 완료
            print("+----+----+----+----+")  
            print("+  0 +  1 +  2 +  3 +") 
            print("+----+----+----+----+") 
            print("+  4 +  5 +  6 +  7 +")  
            print("+----+----+----+----+")  
            print("+  8 +  9 + 10 + 11 +")  
            print("+----+----+----+----+") 
            print("+ 12 + 13 + 14 + 15 +")  
            print("+----+----+----+----+")  
    
            # 키보드로 가능한 행동을 입력 받음
            action = input("Select action(human) : ")
            action = int(action)
            
            # 입력받은 행동이 가능한 행동이면 반복문을 탈출
            if action in available_action:
                return action
            # 아니면 행동 입력을 반복
            else:
                print("You selected wrong action")
        return

4. Actor-Critic 플레이어

In [4]:
class ActorCriticPlayer:
    def __init__(self):
        self.name = "ActorCritic"
        self.V = {}        # 크리틱: 상태 가치 함수 V(s)
        self.policy = {}   # 액터: 상태-행동 확률 정책 ㅠ(s,a)
        self.alpha = 0.1   # 크리틱: 상태 가치 업데이트 속도   ✅ 이상하다싶으면 원상복귀
        self.beta = 0.4    # <- 이전) 0.2   주의) 얘는 유지 바꾸니 변동이 커짐
                           # 액터: 정책 학습률 0.01 -> 0.1->0.2 ->0.4로 수정함
                           #       정책 업데이트 속도 (actor)
                           # TD 오차에 따라 정책 빠르게 업데이트 가능하도록...
        self.gamma = 0.9   # 감가율 : 미래 보상 중요도  (0.93으로 강화해 디버깅해봤을 때 td 에러 더 큼. 따라서 0.9로 유지 )
      
    def get_state_key(self, board):
         # 상태를 딕셔너리 key로 사용하기 위해 튜플로 변환
        return tuple(board.astype(int))

    def softmax(self, probs):
        # Softmax: 확률 분포 계산 (확률값 정규화)
        exp_probs = np.exp(probs - np.max(probs))  # overflow 방지
        return exp_probs / np.sum(exp_probs)

    def select_action(self, env, player):
        # 현재 상태에서 행동 선택
        state_key = self.get_state_key(env.board_a)
        available_actions = env.get_action()

        # 정책이 없다면: 무작위로 초기화하기 (사용 가능한 행동만)
        if state_key not in self.policy:
            self.policy[state_key] = np.zeros(16)
            for i in available_actions:
                self.policy[state_key][i] = np.random.rand()  # 균등 초기화보다 빠르게 수렴 유도

        # 소프트맥스 이전 값 
        logits = self.policy[state_key].copy() #probs 변수명 -> logits로 바꿈 : 소프트맥스 적용 전 값이므로, 좀더 이해하기 편하게 하기 위한 용도 

        
        # 불가능한 행동은 -np.inf로 (액크플레이어가 덮어쓰는 걸 방지하기 위함)
        for i in range(16):
            if i not in available_actions:
                logits[i] = -np.inf
                
        # Gibbs softmax로 확률로 변환하기 (정규화)
        probs = self.softmax(logits)

        # 확률에 따라 행동 선택(Gibbs 정책)
        action = np.random.choice(range(16), p=probs)
        return action

    def update(self, state, action, reward, next_state, done):
        # 학습 단계) Actor-Critic 동시 업데이트하기
        state_key = self.get_state_key(state)
        next_state_key = self.get_state_key(next_state)

        # 상태 가치 함수 및 정책 초기화
        if state_key not in self.V:
            self.V[state_key] = 0.0
        if next_state_key not in self.V:
            self.V[next_state_key] = 0.0
        if state_key not in self.policy:
            self.policy[state_key] = np.zeros(16)
            for i in range(16):
                self.policy[state_key][i] = np.random.rand()
                
        # [Critic] 학습 ----------------------------------------------
        # TD error는 δt=r(t+1)+γV(S(t+1) )-V(St)
        # 1) TD target 계산
        if done:
            td_target = reward # 게임 종료 시: 다음 상태 고려 X
        else:
            td_target = reward + self.gamma * self.V[next_state_key] # td_target
        # 2) TD error
        td_error = td_target - self.V[state_key] # TD 오차 = 실제 - 예측

        # Critic 가치 함수 업데이트 (TD 학습)
        self.V[state_key] += self.alpha * td_error

        # [Actor] 학습 : 정책 업데이트 (TD 오차에 따라 정책 수정) ------
        
        self.policy[state_key][action] += self.beta * td_error

        # 확률 음수 방지 + Softmax 다시 정규화
        self.policy[state_key] = np.maximum(self.policy[state_key], 1e-5)
        self.policy[state_key] = self.softmax(self.policy[state_key])


5. Actor-Critic 플레이어 훈련

In [ ]:
# 학습 에이전트 두 명 생성
p1 = ActorCriticPlayer()
p2 = ActorCriticPlayer()

# 학습 에피소드 수 
# episodes = 30000  # 학습 반복 횟수
# episodes = 70000
episodes  = 200000 # 100000 잠시 대기 

print("Start Actor-Critic Training...")

for episode in tqdm(range(episodes)):
    env = Environment()
    env.print = False  # 학습 시에는 출력 생략

    # 초기 상태 저장
    state = env.board_a.copy()

    for turn in range(100):
        # 현재 플레이어 결정 (1 또는 -1)
        player = 1 if turn % 2 == 0 else -1
        current_player = p1 if player == 1 else p2

        # 행동 선택
        action = current_player.select_action(env, player)

        # 행동 적용
        env.board_a[action] = player

        # 종료 여부 확인
        env.end_check(player)
        
        reward = env.reward
        done = env.done
        next_state = env.board_a.copy()

        if done:
            if reward == player:
                r = 10       # 승리 보상 강화  (반복 끝에 승리보상을 강화하기)
            elif reward == 0:
                r = 1        # 무승부 보상 완화 (10배 강화하니까 비교적 학습 O)
            else:
                r = -10      # 패배 패널티 강화 
        else:
            r = -0.5     # 진행 중 패널티 강화 (-0.05값을 -0.1~1.0으로 조정 실험후 -0.5로 고정)
            

        # 업데이트 수행
        current_player.update(state, action, r, next_state, done)

        # 다음 상태로 전이 
        state = next_state.copy()

        if done:
            break
            
print("Training finished.")

trained_p2 = p2


print("총 학습된 상태 수:", len(p2.V))
print("예시 상태 가치:", list(p2.V.items())[:3])
print("예시 상태 정책:", list(p2.policy.items())[:1])

Start Actor-Critic Training...


 48%|███████████████████████████████████▏                                      | 95094/200000 [07:17<08:08, 214.81it/s]

  6.게임 실행 함수

In [ ]:
np.random.seed(0)

p1 = Human_player()
# p1 = trained_p1
p2 = trained_p2
# p2 = Human_player()

# 지정된 게임 수를 자동으로 두게 할 것인지 한게임씩 두게 할 것인지 결정
# auto = True : 지정된 판수(games)를 자동으로 진행 
# auto = False : 한판씩 진행

auto = False

# auto 모드의 게임수
games = 100

print("pl player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0


if auto: 
    # 자동 모드 실행
    for j in tqdm(range(games)):
        
        np.random.seed(j)
        env = Environment()
        
        for i in range(10000):
            # p1 과 p2가 번갈아 가면서 게임을 진행
            # p1(1) -> p2(-1) -> p1(1) -> p2(-1) ...
            reward, done = env.move(p1,p2,(-1)**i)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    p1_score += 1
                elif reward == -1:
                    p2_score += 1
                else:
                    draw_score += 1
                break

else:                
    # 한 게임씩 진행하는 수동 모드
    np.random.seed(1)
    while True:
        
        env = Environment()
        env.print = False
        for i in range(10000):
            reward, done = env.move(p1,p2,(-1)**i)
            env.print_board()
            if done == True:
                if reward == 1:
                    print("winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("winner is p2({})".format(p2.name))
                    p2_score += 1
                else:
                    print("draw")
                    draw_score += 1
                break
        
        # 최종 결과 출력        
        print("final result")
        env.print_board()

        # 한게임 더?최종 결과 출력 
        answer = input("More Game? (y/n)")

        if answer == 'n':
            break           

print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score,p2.name, p2_score,draw_score))
                